In [1]:
# make sure working directory is root of repository
%cd ..
%pwd

/Users/lydiareader/git/dcds-500-project-2


'/Users/lydiareader/git/dcds-500-project-2'

In [2]:
from shopping_environment import ShoppingEnvironment
import pandas as pd

## Setup Environment

In [3]:
# parameters
num_shoes        = 1000

# number of each consumer type
num_average     = 9000
num_abnormal    = 1000
num_special     = 250
num_wealthy     = 1000
num_influencers = 500
num_resellers   = 2500
num_total       = num_average + num_abnormal + num_special + num_wealthy + num_influencers + num_resellers

# initialize environment
env = ShoppingEnvironment(
    mean_desire = 0.5,
    std_desire = 0.2,
    desire_threshold = 0.5,
    average_mean_money = 250,
    average_std_money = 15,
    rich_mean_money = 1000,
    rich_std_money = 250,
    average_prob_loyal = 0.5,
    rich_prob_loyal = 0.8,
    reseller_prob_loyal = 0.9,
    num_shoes = num_shoes,
    price = 250,
    num_average = num_average,
    num_abnormal = num_abnormal,
    num_special = num_special,
    num_wealthy = num_wealthy,
    num_influencers = num_influencers,
    num_resellers = num_resellers
)

In [4]:
def get_utilities(df):
    # buyer and seller utility for each simulation
    got_shoes = df[df['shoes_acquired'] >= 1]
    print("Consumer utility: ", got_shoes['desire'].mean())     # buyer utility is average desire of those who purchased shoes
    print("Seller utility: ", got_shoes['influence'].mean())    # seller utility is average influence of those who purchased shoes 


def group_fairness(df):
    ret_dict = {}
    got_shoes = df[df['shoes_acquired'] >= 1]
    ret_dict['Consumer utility'] = got_shoes['desire'].mean()
    ret_dict['Seller utility'] = got_shoes['influence'].mean()
    amount_con_types = {
        'average': num_average,
        'abnormal': num_abnormal,
        'special': num_special,
        'wealthy': num_wealthy,
        'influencer': num_influencers,
        'reseller': num_resellers
    }
    for con, amount in amount_con_types.items():
        ret_dict[f'{con}_num_got_shoes'] = len(got_shoes[got_shoes['identity'] == con])
        ret_dict[f'{con}_percent_got_shoes'] = len(got_shoes[got_shoes['identity'] == con]) / amount
        ret_dict[f'{con}_shoes_bought'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum()
        ret_dict[f'{con}_shoes_per_person'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum() / amount

    return ret_dict
        

## No Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [5]:
env.restock(1000)
env.run_lottery_no_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_no_gaming = pd.Series(group_fairness(df), name='lottery_no_gaming')
lottery_no_gaming

Consumer utility                  0.397283
Seller utility                    0.133600
average_num_got_shoes           397.000000
average_percent_got_shoes         0.044111
average_shoes_bought            591.000000
average_shoes_per_person          0.065667
abnormal_num_got_shoes           46.000000
abnormal_percent_got_shoes        0.046000
abnormal_shoes_bought            67.000000
abnormal_shoes_per_person         0.067000
special_num_got_shoes            15.000000
special_percent_got_shoes         0.060000
special_shoes_bought             30.000000
special_shoes_per_person          0.120000
wealthy_num_got_shoes            35.000000
wealthy_percent_got_shoes         0.035000
wealthy_shoes_bought             70.000000
wealthy_shoes_per_person          0.070000
influencer_num_got_shoes         22.000000
influencer_percent_got_shoes      0.044000
influencer_shoes_bought          22.000000
influencer_shoes_per_person       0.044000
reseller_num_got_shoes          110.000000
reseller_pe

### First Come First Served
With no gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [7]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_no_gaming()
df = env.get_consumer_df()
first_no_gaming = pd.Series(group_fairness(df), name='first_no_gaming')
first_no_gaming

Consumer utility                  0.900474
Seller utility                    0.056054
average_num_got_shoes           534.000000
average_percent_got_shoes         0.059333
average_shoes_bought            783.000000
average_shoes_per_person          0.087000
abnormal_num_got_shoes           52.000000
abnormal_percent_got_shoes        0.052000
abnormal_shoes_bought            77.000000
abnormal_shoes_per_person         0.077000
special_num_got_shoes             8.000000
special_percent_got_shoes         0.032000
special_shoes_bought             16.000000
special_shoes_per_person          0.064000
wealthy_num_got_shoes            49.000000
wealthy_percent_got_shoes         0.049000
wealthy_shoes_bought             98.000000
wealthy_shoes_per_person          0.098000
influencer_num_got_shoes         26.000000
influencer_percent_got_shoes      0.052000
influencer_shoes_bought          26.000000
influencer_shoes_per_person       0.052000
reseller_num_got_shoes            0.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [9]:
env.restock(num_shoes = 1000)
env.run_invitation_no_gaming()
df = env.get_consumer_df()
invite_no_gaming = pd.Series(group_fairness(df), name='invite_no_gaming')
invite_no_gaming

Consumer utility                  0.461696
Seller utility                    0.364090
average_num_got_shoes           198.000000
average_percent_got_shoes         0.022000
average_shoes_bought            298.000000
average_shoes_per_person          0.033111
abnormal_num_got_shoes           15.000000
abnormal_percent_got_shoes        0.015000
abnormal_shoes_bought            24.000000
abnormal_shoes_per_person         0.024000
special_num_got_shoes             5.000000
special_percent_got_shoes         0.020000
special_shoes_bought             10.000000
special_shoes_per_person          0.040000
wealthy_num_got_shoes            30.000000
wealthy_percent_got_shoes         0.030000
wealthy_shoes_bought             60.000000
wealthy_shoes_per_person          0.060000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes           54.000000
reseller_pe

In [8]:
no_gaming = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming])

## With Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [10]:
env.restock(1000)
env.run_lottery_with_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_with_gaming = pd.Series(group_fairness(df), name='lottery_with_gaming')
lottery_with_gaming

Consumer utility                  0.168591
Seller utility                    0.356855
average_num_got_shoes           128.000000
average_percent_got_shoes         0.014222
average_shoes_bought            214.000000
average_shoes_per_person          0.023778
abnormal_num_got_shoes           12.000000
abnormal_percent_got_shoes        0.012000
abnormal_shoes_bought            21.000000
abnormal_shoes_per_person         0.021000
special_num_got_shoes             2.000000
special_percent_got_shoes         0.008000
special_shoes_bought              4.000000
special_shoes_per_person          0.016000
wealthy_num_got_shoes            28.000000
wealthy_percent_got_shoes         0.028000
wealthy_shoes_bought             55.000000
wealthy_shoes_per_person          0.055000
influencer_num_got_shoes          2.000000
influencer_percent_got_shoes      0.004000
influencer_shoes_bought           2.000000
influencer_shoes_per_person       0.004000
reseller_num_got_shoes          324.000000
reseller_pe

### First Come First Served
With gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [11]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_with_gaming()

df = env.get_consumer_df()
first_with_gaming = pd.Series(group_fairness(df), name='first_with_gaming')
first_with_gaming

Consumer utility                  0.793596
Seller utility                    0.192790
average_num_got_shoes           180.000000
average_percent_got_shoes         0.020000
average_shoes_bought            274.000000
average_shoes_per_person          0.030444
abnormal_num_got_shoes           14.000000
abnormal_percent_got_shoes        0.014000
abnormal_shoes_bought            21.000000
abnormal_shoes_per_person         0.021000
special_num_got_shoes             2.000000
special_percent_got_shoes         0.008000
special_shoes_bought              4.000000
special_shoes_per_person          0.016000
wealthy_num_got_shoes            52.000000
wealthy_percent_got_shoes         0.052000
wealthy_shoes_bought            104.000000
wealthy_shoes_per_person          0.104000
influencer_num_got_shoes         12.000000
influencer_percent_got_shoes      0.024000
influencer_shoes_bought          12.000000
influencer_shoes_per_person       0.024000
reseller_num_got_shoes           59.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [12]:
env.restock(num_shoes = 1000)
env.run_invitation_with_gaming()
df = env.get_consumer_df()
invite_with_gaming = pd.Series(group_fairness(df), name='invite_with_gaming')
invite_with_gaming

Consumer utility                  0.359730
Seller utility                    0.479839
average_num_got_shoes            26.000000
average_percent_got_shoes         0.002889
average_shoes_bought             52.000000
average_shoes_per_person          0.005778
abnormal_num_got_shoes            3.000000
abnormal_percent_got_shoes        0.003000
abnormal_shoes_bought             6.000000
abnormal_shoes_per_person         0.006000
special_num_got_shoes             1.000000
special_percent_got_shoes         0.004000
special_shoes_bought              2.000000
special_shoes_per_person          0.008000
wealthy_num_got_shoes            10.000000
wealthy_percent_got_shoes         0.010000
wealthy_shoes_bought             20.000000
wealthy_shoes_per_person          0.020000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes          204.000000
reseller_pe

## Change cap to 1
Change the cap to one to prevent resellers get too many shoes and ensure the fairness

In [13]:
env.restock(1000)
env.run_lottery_with_gaming(cap=1)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_cap1_with_gaming')
lottery_cap1_with_gaming

Consumer utility                  0.174616
Seller utility                    0.360400
average_num_got_shoes           232.000000
average_percent_got_shoes         0.025778
average_shoes_bought            235.000000
average_shoes_per_person          0.026111
abnormal_num_got_shoes           19.000000
abnormal_percent_got_shoes        0.019000
abnormal_shoes_bought            19.000000
abnormal_shoes_per_person         0.019000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            43.000000
wealthy_percent_got_shoes         0.043000
wealthy_shoes_bought             44.000000
wealthy_shoes_per_person          0.044000
influencer_num_got_shoes         15.000000
influencer_percent_got_shoes      0.030000
influencer_shoes_bought          15.000000
influencer_shoes_per_person       0.030000
reseller_num_got_shoes          590.000000
reseller_pe

## Authentication

With auth, it's harder for reseller get fake acccounts

In [14]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=2)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_authen_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_with_gaming')
lottery_authen_with_gaming

Consumer utility                  0.184519
Seller utility                    0.334906
average_num_got_shoes           154.000000
average_percent_got_shoes         0.017111
average_shoes_bought            239.000000
average_shoes_per_person          0.026556
abnormal_num_got_shoes           17.000000
abnormal_percent_got_shoes        0.017000
abnormal_shoes_bought            26.000000
abnormal_shoes_per_person         0.026000
special_num_got_shoes             4.000000
special_percent_got_shoes         0.016000
special_shoes_bought              8.000000
special_shoes_per_person          0.032000
wealthy_num_got_shoes            16.000000
wealthy_percent_got_shoes         0.016000
wealthy_shoes_bought             32.000000
wealthy_shoes_per_person          0.032000
influencer_num_got_shoes          7.000000
influencer_percent_got_shoes      0.014000
influencer_shoes_bought           7.000000
influencer_shoes_per_person       0.014000
reseller_num_got_shoes          332.000000
reseller_pe

In [16]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=1)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_authen_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_cap1_with_gaming')
lottery_authen_cap1_with_gaming

Consumer utility                  0.203462
Seller utility                    0.331176
average_num_got_shoes           283.000000
average_percent_got_shoes         0.031444
average_shoes_bought            283.000000
average_shoes_per_person          0.031444
abnormal_num_got_shoes           30.000000
abnormal_percent_got_shoes        0.030000
abnormal_shoes_bought            30.000000
abnormal_shoes_per_person         0.030000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            33.000000
wealthy_percent_got_shoes         0.033000
wealthy_shoes_bought             33.000000
wealthy_shoes_per_person          0.033000
influencer_num_got_shoes         23.000000
influencer_percent_got_shoes      0.046000
influencer_shoes_bought          23.000000
influencer_shoes_per_person       0.046000
reseller_num_got_shoes          558.000000
reseller_pe

## Combine three distribution methods

Since it's easier for resellers to use bot online, shoes will be distributed in a proportion that fewer percentage of them can be bought online to prevent that

In [18]:
env.restock(200)
env.run_lottery_with_gaming_auth()
env.restock_without_reset(500)
env.run_first_come_with_gaming()
env.restock_without_reset(300)
env.run_invitation_with_gaming()
df = env.get_consumer_df()
combined_distribution_with_gaming = pd.Series(group_fairness(df), name='combined_distribution_with_gaming')
combined_distribution_with_gaming

Consumer utility                  0.557594
Seller utility                    0.368020
average_num_got_shoes           141.000000
average_percent_got_shoes         0.015667
average_shoes_bought            214.000000
average_shoes_per_person          0.023778
abnormal_num_got_shoes           15.000000
abnormal_percent_got_shoes        0.015000
abnormal_shoes_bought            23.000000
abnormal_shoes_per_person         0.023000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            37.000000
wealthy_percent_got_shoes         0.037000
wealthy_shoes_bought             75.000000
wealthy_shoes_per_person          0.075000
influencer_num_got_shoes        309.000000
influencer_percent_got_shoes      0.618000
influencer_shoes_bought         309.000000
influencer_shoes_per_person       0.618000
reseller_num_got_shoes           89.000000
reseller_pe

## Get Final Results

In [19]:
final_result = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming,lottery_with_gaming,first_with_gaming,invite_with_gaming,lottery_cap1_with_gaming,lottery_authen_with_gaming,lottery_authen_cap1_with_gaming,combined_distribution_with_gaming])

In [20]:
# uncomment to save results
# final_result.to_csv('project2_results.txt',sep = '\t') 
final_result

,Consumer utility,Seller utility,average_num_got_shoes,average_percent_got_shoes,average_shoes_bought,average_shoes_per_person,abnormal_num_got_shoes,abnormal_percent_got_shoes,abnormal_shoes_bought,abnormal_shoes_per_person,...,wealthy_shoes_bought,wealthy_shoes_per_person,influencer_num_got_shoes,influencer_percent_got_shoes,influencer_shoes_bought,influencer_shoes_per_person,reseller_num_got_shoes,reseller_percent_got_shoes,reseller_shoes_bought,reseller_shoes_per_person
lottery_no_gaming,0.397283,0.133600,397.0,0.044111,591.0,0.065667,46.0,0.046,67.0,0.067,...,70.0,0.070,22.0,0.044,22.0,0.044,110.0,0.0440,220.0,0.0880
first_no_gaming,0.900474,0.056054,534.0,0.059333,783.0,0.087000,52.0,0.052,77.0,0.077,...,98.0,0.098,26.0,0.052,26.0,0.052,0.0,0.0000,0.0,0.0000
invite_no_gaming,0.461696,0.364090,198.0,0.022000,298.0,0.033111,15.0,0.015,24.0,0.024,...,60.0,0.060,500.0,1.000,500.0,1.000,54.0,0.0216,108.0,0.0432
lottery_with_gaming,0.168591,0.356855,128.0,0.014222,214.0,0.023778,12.0,0.012,21.0,0.021,...,55.0,0.055,2.0,0.004,2.0,0.004,324.0,0.1296,704.0,0.2816
first_with_gaming,0.793596,0.192790,180.0,0.020000,274.0,0.030444,14.0,0.014,21.0,0.021,...,104.0,0.104,12.0,0.024,12.0,0.024,59.0,0.0236,585.0,0.2340
invite_with_gaming,0.359730,0.479839,26.0,0.002889,52.0,0.005778,3.0,0.003,6.0,0.006,...,20.0,0.020,500.0,1.000,500.0,1.000,204.0,0.0816,420.0,0.1680
lottery_cap1_with_gaming,0.174616,0.360400,232.0,0.025778,235.0,0.026111,19.0,0.019,19.0,0.019,...,44.0,0.044,15.0,0.030,15.0,0.030,590.0,0.2360,677.0,0.2708
lottery_authen_with_gaming,0.184519,0.334906,154.0,0.017111,239.0,0.026556,17.0,0.017,26.0,0.026,...,32.0,0.032,7.0,0.014,7.0,0.014,332.0,0.1328,688.0,0.2752
lottery_authen_cap1_with_gaming,0.203462,0.331176,283.0,0.031444,283.0,0.031444,30.0,0.030,30.0,0.030,...,33.0,0.033,23.0,0.046,23.0,0.046,558.0,0.2232,619.0,0.2476
combined_distribution_with_gaming,0.557594,0.368020,141.0,0.015667,214.0,0.023778,15.0,0.015,23.0,0.023,...,75.0,0.075,309.0,0.618,309.0,0.618,89.0,0.0356,379.0,0.1516
